In [ ]:
import modulos as md

#lista=md.filtradodemonedas()

lista = ['BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'XLMUSDT', 'ADAUSDT', 'BNBUSDT', 'ATOMUSDT', 'ALGOUSDT', 
         'KNCUSDT', 'COMPUSDT', 'DOGEUSDT', 'KAVAUSDT', 'WAVESUSDT', 'MKRUSDT', 'SNXUSDT', 'DOTUSDT', 'CRVUSDT', 'SOLUSDT', 'STORJUSDT', 'UNIUSDT', 
         'AVAXUSDT', 'FTMUSDT', 'TOMOUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'MATICUSDT', 'AXSUSDT', 'SANDUSDT', 'XEMUSDT', 'MANAUSDT', 'LINAUSDT', 
         'MTLUSDT', '1000SHIBUSDT', 'MASKUSDT', 'DYDXUSDT', '1000XECUSDT', 'GALAUSDT', 'FLOWUSDT', 'GMTUSDT', 'APEUSDT', 'OPUSDT', 'INJUSDT', 'LDOUSDT', 
         'APTUSDT', 'FETUSDT', 'RNDRUSDT', 'AGIXUSDT', 'PHBUSDT', 'CFXUSDT', 'STXUSDT', 'LQTYUSDT', 'ARBUSDT', 'SUIUSDT', '1000PEPEUSDT', 'XVGUSDT']

#lista=['SKLUSDT']

In [ ]:
###################################### FILTRADO DE MONEDAS PARA ESTRATEGIA SANTAAAAAA
###################################################################################################################
for symbol in lista:
    try:
        data = md.estrategia_santa(symbol, tp_flag = True)
        resultado = md.backtesting(data, plot_flag = False)
        #if (resultado['Profit Factor']>1 or resultado['Return [%]'] >0) and resultado['Win Rate [%]']>50:
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)} - Win Rate [%]: {md.truncate(resultado['Win Rate [%]'],2)}")
    except Exception as falla:
        pass

In [ ]:
###################################### FILTRADO DE MONEDAS PARA ESTRATEGIA BB
###################################################################################################################
for symbol in lista:
    try:
        data = md.estrategia_bb(symbol, tp_flag = True)
        resultado = md.backtesting(data, plot_flag = False)
        #if resultado['Profit Factor'] > 1 and resultado[6]/resultado[17] >=0.3:
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)}")
    except Exception as falla:
        pass

In [1]:
import modulos as md
###################################### variacion btc
###################################################################################################################
lista = ['BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'XLMUSDT', 'ADAUSDT', 'BNBUSDT', 'ATOMUSDT', 'ALGOUSDT', 
         'KNCUSDT', 'COMPUSDT', 'DOGEUSDT', 'KAVAUSDT', 'WAVESUSDT', 'MKRUSDT', 'SNXUSDT', 'DOTUSDT', 'CRVUSDT', 'SOLUSDT', 'STORJUSDT', 'UNIUSDT', 
         'AVAXUSDT', 'FTMUSDT', 'TOMOUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'MATICUSDT', 'AXSUSDT', 'SANDUSDT', 'XEMUSDT', 'MANAUSDT', 'LINAUSDT', 
         'MTLUSDT', '1000SHIBUSDT', 'MASKUSDT', 'DYDXUSDT', '1000XECUSDT', 'GALAUSDT', 'FLOWUSDT', 'GMTUSDT', 'APEUSDT', 'OPUSDT', 'INJUSDT', 'LDOUSDT', 
         'APTUSDT', 'FETUSDT', 'RNDRUSDT', 'AGIXUSDT', 'PHBUSDT', 'CFXUSDT', 'STXUSDT', 'LQTYUSDT', 'ARBUSDT', 'SUIUSDT', '1000PEPEUSDT', 'XVGUSDT']
for symbol in lista:
    try:
        data = md.sigo_variacion_bitcoin(symbol,timeframe='15m',porc=0.8,ventana=2,tp_flag=True)
        resultado = md.backtesting(data, plot_flag = False)
        #if resultado['Profit Factor'] > 1 and resultado[6]/resultado[17] >=0.3:
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)}")
    except Exception as falla:
        pass

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


XRPUSDT - Return [%]: 14.86% - # Trades: 15 - Profit Factor: 2.35
EOSUSDT - Return [%]: 7.64% - # Trades: 20 - Profit Factor: 2.62


In [1]:
import modulos as md
import pandas_ta as ta
symbol='XRPUSDT'
data = md.sigo_variacion_bitcoin(symbol,timeframe='15m',porc=0.8,ventana=2,tp_flag=True)
resultado = md.backtesting(data, plot_flag = False)

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Short orders require: TP (nan) < LIMIT (0.4754) < SL (nan)

In [3]:
resultado

Start                     2023-07-05 08:45:00
End                       2023-07-15 18:30:00
Duration                     10 days 09:45:00
Exposure Time [%]                        16.2
Equity Final [$]                  1148.654348
Equity Peak [$]                   1196.893187
Return [%]                          14.865435
Buy & Hold Return [%]               48.547463
Return (Ann.) [%]                10624.453341
Volatility (Ann.) [%]            12329.333719
Sharpe Ratio                         0.861722
Sortino Ratio                     3109.306689
Calmar Ratio                      2636.119795
Max. Drawdown [%]                   -4.030338
Avg. Drawdown [%]                   -1.581544
Max. Drawdown Duration        7 days 01:15:00
Avg. Drawdown Duration        1 days 16:53:00
# Trades                                   15
Win Rate [%]                        46.666667
Best Trade [%]                       9.694789
Worst Trade [%]                     -4.575481
Avg. Trade [%]                    